# Een dataframe verkennen
## Business vraag
Welke producten die in België verkocht worden, bevatten Vitamine D en waar worden die verkocht?
## Download de data
Wanneer we de data downloaden, krijgen we een zipbestand dat we kunnen uitpakken. 

In [ ]:
from pathlib import Path
import requests
FOOD_ZIP = 'food_facts.zip'
food_path = Path(FOOD_ZIP)
if not food_path.exists():
    data = requests.get('https://www.kaggle.com/api/v1/datasets/download/openfoodfacts/world-food-facts')
    with open(FOOD_ZIP, mode='wb') as f:
        f.write(data.content)
else:
    print(f"Bestand {FOOD_ZIP} is al gedownload")

In [ ]:
from pathlib import Path
from zipfile import ZipFile
FOOD_TSV = 'en.openfoodfacts.org.products.tsv'
food_tsv_path = Path(FOOD_TSV)
if not food_tsv_path.exists():
    with open(FOOD_ZIP, mode='rb') as f:
        ZipFile(f).extractall()
else:
    print(f"Bestand {FOOD_TSV} bestaat al")

## Wat voor bestand is het
Ik vind het een goed idee om de eerste regels met Python te lezen. Hier zien we dat het een .tsv-bestand is: een tab-separated-values bestand (dat hadden we ook kunnen zien aan de extensie natuurlijk)

In [ ]:
FOOD_TSV = 'en.openfoodfacts.org.products.tsv'
with open(FOOD_TSV) as f:
    for _ in range(5):
        print(f.readline(), end='')

## Data inlezen met pandas
We geven als delimiter '\t' mee. Bij het inlezen krijgen we een warning. Pandas heeft geprobeerd om de datatypes te 'raden'. Maar omdat het een heel groot bestand is, geven we de types best zelf mee. Om voor elke kolom de datatypes te kennen, moet het volledige bestand eerst worden ingelezen. Pas dan is het datatype gekend. En dat vraag veel meer geheugen.  

In [ ]:
import pandas as pd
FOOD_TSV = 'en.openfoodfacts.org.products.tsv'
df = pd.read_csv(FOOD_TSV, delimiter='\t', index_col='code')
df.head()

## Hoeveel rijen zijn er?
De shape van een dataframe bevat het aantal rijen en kolommen. (denk aan NumPy)

In [ ]:
df.shape[0]

## Welke kolommen hebben we?
Met de info()-functie kunnen we informatie opvragen over het dataframe, maar in dit geval is de informatie zo uitgebreid (163 kolommen) dat we niets te zien krijgen.

In [ ]:
df.info()

## .info(verbose=True)
Met het argument 'verbose' kunnen we meer informatie zien. (kies: 'view as scrollable element'). 

In [ ]:
df.info(verbose=True)

## We vertrekken van een Business vraag
Welke voedingsmiddelen in België bevatten vitamine d ? Welke kolommen hebben we daarvoor nodig? 
countries_en, product_name, brands, stores, vitamin-d_100g. We geven meteen ook de types mee. (let op het type voor Code.)


In [ ]:
import pandas as pd
FOOD_TSV = 'en.openfoodfacts.org.products.tsv'
df_data = pd.read_csv(FOOD_TSV, delimiter='\t', index_col='code', 
                      usecols=['code', 'countries_en', 'product_name','brands', 'stores', 'vitamin-d_100g'],
                      dtype={'code':pd.StringDtype(), 'countries_en':pd.StringDtype(), 
                             'product_name':pd.StringDtype(), 'brands':pd.StringDtype(),
                             'stores':pd.StringDtype(),'vitamin-d_100g':pd.Float64Dtype()})
df_data.info()

## Filter NA-waarden voor 'countries_en' en 'vitamin-d_100g'
Aangezien we op zoek zijn naar waarden voor Belgium  en vitamine D heeft het niet veel zin om records met lege waarden voor deze kolommen te behouden. We zullen die verwijderen. Hiervoor kunnen we .dropna() gebruiken. We beperken de controle tot de kolommen *countries_en* en *vitamin-d_100g*.

In [ ]:
df_data = df_data.dropna(subset=['countries_en', 'vitamin-d_100g'])
df_data.info()

## Hoe ziet de countries_en kolom er uit?
Bevat die kolom 1 landnaam of meerdere?

In [ ]:
pd.set_option('display.max_rows', 100) # 100 is de standaardwaarde maar we kunnen dit wijzigen
for item in df_data['countries_en']:
    print(item)

## Worden alle producten alleen in Belgie verkocht?
In NumPy hebben we np.strings om de gevectoriseerde string-functies te gebruiken. In Pandas hebben we iets gelijkaardig via DataFrame.str.

We kunnen nu uitzoeken of België altijd alleen voorkomt of soms ook in combinatie met anderen landen?

In [ ]:
alleen_belgie = df_data[df_data['countries_en']=='Belgium']
belgie_gecombineerd = df_data[df_data['countries_en'].str.contains('Belgium')]
print(len(alleen_belgie))
print(len(belgie_gecombineerd))

## We willen alleen producten overhouden met waarden België
We werken verder met de filter belgie_gecombineerd 

In [ ]:
df_data = df_data[df_data['countries_en'].str.contains('Belgium')]

df_data.info()

## Wat zijn de productnamen voor die 39 producten en wie verkoopt ze?
We zullen de gegevens sorteren op basis van 'vitamin-d_100g' (van groot naar klein). De iterator voor de dataset krijgen we met iterrows. Die geeft (index, rij) terug:

In [ ]:
for _, merk in df_data.loc[:, ['product_name', 'brands', 'stores', 'vitamin-d_100g']].sort_values('vitamin-d_100g', ascending=False).iterrows():
    print(merk.at['product_name'], merk.at['brands'], merk.at['stores'],merk.at['vitamin-d_100g'])

## Conclusie
Blijkbaar zit er 'relatief veel' vitamine d in Nesquik. We kunnen dit plotten in een horizontale barchart. Omdat de aantallen verschillen in de grootte-orde van machten van 10, gebruiken we een log-schaal.

Om de 10 hoogste waarden te pakken te krijgen, vragen we eerst de indexen op van die waarden. Vervolgens gebruiken we dat om de productnaam en het vitamine D gehalte over te houden. 

We gebruiken *gca().invert_yaxis()* (*gca=get current axes*) om de waarden van groot naar klein te tonen. 

In [ ]:
import matplotlib.pyplot as plt
indexen = df_data.sort_values(by='vitamin-d_100g', ascending=False).index[:10]
plt_data = df_data.loc[indexen, ['product_name', 'vitamin-d_100g']]
plt.barh(plt_data['product_name'], plt_data['vitamin-d_100g'], log=True)
plt.xlabel('log(vitamine D per 100 g)')
plt.gca().invert_yaxis()
plt.title('Vitamine D in producten')
plt.show()